In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py
import csv
import sys

import mass_value_calculator
import strain_calculator
import cosmology_calculator_simple

In [7]:
# file name
fname = 'holodeck_illustris_pop-r10.hdf5'

# which universe
real = '0009'
#real = str(sys[1])

# constants
MSOL = 1.9884099e33           # Solar mass in grams
G = 4.5170e-48   # Mpc^3 M_solar^-1 s^-2
c = 9.7146e-15   # Mpc s^-1



In [8]:
# initilize arrays
chirp_mass_r = [] # Solar Masses, rest frame
chirp_mass_o = [] # Solar Masses, observor frame
d_l = []          # Mpc, luminosity distance
d_c = []          # Mpc, comoving distance
h_o = []
h_s = []
f_r = []          # Hz, rest-frame orbital frequency


In [9]:
# read data from file
with h5py.File(fname, 'r') as data:
        
    # extract data from one realizations
    real_data = data[real]
        
    z = real_data['redz'][()]
        
    q = real_data['mrat'][()]
    
    mtot = real_data['mtot'][()] # grams
        
    f_o = real_data['fobs'][()] # Hz , observor frame GW frequency 
    
#z = z[0:10]
#q = q[0:10]
#mtot = mtot[0:10]
#f_o = f_o[0:10]
     

In [10]:
for w in range(len(z)):
    # calculate luminostiy distance and comoving distance
    # input: (z)
    # output: (z, DCMR_Mpc, DL_Mpc, kpc_DA)
    distances = cosmology_calculator_simple.cosmo_calc(z[w])
    d_l.append(distances[2])
    d_c.append(distances[1])
    
    # calculate chirp mass in rest frame and observor frame
    # input: (m1, m2, Mtot, q, Mc, mu) (all solar masses)
    # output: (m1, m2, Mtot, q, Mc, mu)
    masses = mass_value_calculator.mass_val_calc(None, None, float(mtot[w] / MSOL), float(q[w]), None, None)
    chirp_mass_r.append(masses[4])
    chirp_mass_o.append(masses[4] * (1 + z[w]))
        
    # calculate strain
    # input: strain_calc(chirp mass (Solar Masses), Dl (Mpc), GW f, (Hz))
    # output: strain
    strain = strain_calculator.strain_calc(chirp_mass_o[w], d_l[w], f_o[w])
    h_o.append(strain)
    
    # calculate rest frame frequency
    f_r.append((f_o[w] * (1 + z[w])) / 2)
    
    # calculate h_s
    strain = (8 / 10 ** (1/2)) * (G * chirp_mass_r[w])**(5/3) * (2 * np.pi * f_r[w])**(2/3) / (c**4 * d_c[w]) 
    h_s.append(strain)



In [11]:
# characteristic strain for a particular source

# bin 1: 5.4e-9, 1.04e-8
# bin 2: 1.04e-8, 1.55e-8
# bin 3: 1.55e-8, 2.13e-8
# bin 4: 2.13e-8, 2.64e-8

bin1_cen = 7.9e-9
bin2_cen = 1.33e-8
bin3_cen = 1.8653e-8
bin4_cen = 2.38e-8

delta_f = 5e-9

h_sc = np.zeros(len(h_s))

h_c_1_sqed = 0

h_c_2_sqed = 0

h_c_3_sqed = 0

h_c_4_sqed = 0


for i in range(len(h_s)):
    
    if f_o[i] >= 5.4e-9 and f_o[i] < 1.04e-8:
        h_sc[i] = h_s[i] * np.sqrt(bin1_cen / delta_f)
        
        h_c_1_sqed += h_s[i]**2 * (bin1_cen / delta_f)
        
        
    if f_o[i] >= 1.04e-9 and f_o[i] < 1.55e-8:
        h_sc[i] = h_s[i] * np.sqrt(bin2_cen / delta_f)
        
        h_c_2_sqed += h_s[i]**2 * (bin2_cen / delta_f)
        
    if f_o[i] >= 1.55e-8 and f_o[i] < 2.13e-8:
        h_sc[i] = h_s[i] * np.sqrt(bin3_cen / delta_f)
        
        h_c_3_sqed += h_s[i]**2 * (bin3_cen / delta_f)
        
    if f_o[i] >= 2.13e-8 and f_o[i] < 2.64e-8:
        h_sc[i] = h_s[i] * np.sqrt(bin4_cen / delta_f)
        
        h_c_4_sqed += h_s[i]**2 * (bin4_cen / delta_f)
        
background = [np.sqrt(h_c_1_sqed), np.sqrt(h_c_2_sqed), np.sqrt(h_c_3_sqed), np.sqrt(h_c_4_sqed)]
centers = [bin1_cen, bin2_cen, bin3_cen, bin4_cen]

In [12]:
# highest strain in each bin
bin1_source = 0
bin2_source = 0
bin3_source = 0
bin4_source = 0


for i in range(len(h_s)):
    
    # bin 1
    if f_o[i] >= 5.4e-9 and f_o[i] < 1.04e-8:
        if h_sc[i] > bin1_source:
            bin1_source = h_sc[i]
        
    # bin 2    
    if f_o[i] >= 1.04e-9 and f_o[i] < 1.55e-8:
        if h_sc[i] > bin2_source:
            bin2_source = h_sc[i]
       
    # bin 3    
    if f_o[i] >= 1.55e-8 and f_o[i] < 2.13e-8:
        if h_sc[i] > bin3_source:
            bin3_source = h_sc[i]
      
    # bin 4    
    if f_o[i] >= 2.13e-8 and f_o[i] < 2.64e-8:
        if h_sc[i] > bin4_source:
            bin4_source = h_sc[i]


brightest_sources = [bin1_source, bin2_source, bin3_source, bin4_source]

In [14]:
# write data to a file
filename = real + '.csv'
header = ['Chirp_Mass_rest', 'd_l', 'h_o', 'Tot_M', 'f_o', 'q', 'z', 'd_c', 'Chirp_mass_o', 'h_s', 'f_r', 'h_sc']

with open(filename, 'w') as f:
    writer = csv.writer(f)
    
    writer.writerow(header)
    
    for i in range(len(h_o)):
        writer.writerow([chirp_mass_r[i], d_l[i], h_o[i], mtot[i] / MSOL, f_o[i], 
                         q[i], z[i], d_c[i], chirp_mass_o[i], h_s[i], f_r[i], h_sc[i]])

In [15]:
# write background and brightest sources to a file
filename = 'source_background' + real + '.csv'
header = ['brightest_source', 'background']

with open(filename, 'w') as f:
    writer = csv.writer(f)
    
    writer.writerow(header)
    
    for i in range(len(background)):
        writer.writerow([brightest_sources[i], background[i]])
